In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import h5py

In [2]:
# dataset can be found on 'Week 2/Datasets' directory
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [4]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

In [5]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

In [6]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

In [7]:
print('train_set_x shape:',train_set_x_flatten.shape)
print('train_set_y shape:',train_set_y.shape)
print('test_set_x shape:',test_set_x_flatten.shape)
print('test_set_y shape:',test_set_y.shape)
print('classes:',end=' ')
for i in classes:
    print(i,end=', ')

train_set_x shape: (12288, 209)
train_set_y shape: (1, 209)
test_set_x shape: (12288, 50)
test_set_y shape: (1, 50)
classes: b'non-cat', b'cat', 

In [8]:
def sigmoid(z):

    return 1/(1+np.exp(z*(-1)))

In [9]:
def propagate(w,b,X,y):

    m=X.shape[1]

    # forward propagate
    z=np.dot(w.T,X)+b
    y_cap=sigmoid(z)

    # backward propagate
    dz=y_cap-y
    dw=(np.dot(X,dz.T))/m
    db=np.sum(dz)/m

    return dw,db

In [10]:
def optimize(w,b,X,y,num_iteration,alpha):

    for i in range(num_iteration):
        dw,db=propagate(w,b,X,y)

        w-=alpha*dw
        b-=alpha*db

    return w,b

In [11]:
def predict(w,b,X):

    m=X.shape[1]
    y_predicted=sigmoid(np.dot(w.T,X)+b)

    for i in range(y_predicted.shape[1]):
        if y_predicted[0][i]<=0.5:
            y_predicted[0][i]=0
        else:
            y_predicted[0][i]=1

    return y_predicted

In [12]:
def model(X_train,y_train,X_test,num_iteration=2000,alpha=0.5):

    n=X_train.shape[0]
    w=np.zeros((n,1))
    b=0
    
    w_final,b_final=optimize(w,b,X_train,y_train,num_iteration,alpha)
    y_prediction=predict(w_final,b_final,X_test)

    return y_prediction

In [13]:
def checkAccuracy(y_cap,y):

    score=np.mean(np.abs(y_cap-y))
    accuracy=(1-score)*100
    
    return accuracy

In [14]:
y_predicted=model(train_set_x,train_set_y,train_set_x,2000,0.005)
accuracy=checkAccuracy(y_predicted,train_set_y)
print('Train accuracy:',(int(accuracy*100))/100.0,'%')

Train accuracy: 99.04 %


In [15]:
y_predicted=model(train_set_x,train_set_y,test_set_x,2000,0.005)
accuracy=checkAccuracy(y_predicted,test_set_y)
print('Test accuracy:',(int(accuracy*100))/100.0,'%')

Test accuracy: 70.0 %
